In [1]:
#sqrt(1+sqr(x)*sqr(sin(sqr(x)))) where -1<=x<=1

# Importing

In [2]:
import numpy as np
import math
from math import sqrt
from math import pow
from math import sin
from math import cos
from scipy.interpolate import lagrange
eps = 0.0000001

## Segment splitting

In [3]:
def split(a, b, n): # a & b is a segment, n is an amount of fragments
    lengthOfSegment = b-a
    lengthOfFragment = lengthOfSegment / (n-1) #last point should be equal to b
    splitPoints = np.zeros(n) 
    for i in range(n):
        splitPoints[i] = a + i * lengthOfFragment
    return splitPoints
  ################## Example      
split(1, 20, 5)

array([ 1.  ,  5.75, 10.5 , 15.25, 20.  ])

## Here's my function hardcoded

In [4]:
def calcFunValue(value): #Calculates values of calcFunValue
    x = value
    a = -1
    b = 1
    if (value < a) or (value > b):
        return None
    return sqrt(1 + pow(x, 2) * pow(sin(pow(x, 2)), 2))
################## Example
print(calcFunValue(1.1))

None


# Interpolation

## Calculates values of function hardcoded above into nodes values

In [5]:
def valuesInPonts(points): #Calculates values of calcFunValue in points array
    values = np.zeros(len(points))
    for i in range(len(points)):
        values[i] = calcFunValue(points[i])
    return values

## My own lagrange implementation

In [6]:
#Evaluates te i-th Lagrange polynomial at x
def my_lagrange (x ,i , xm ):
    y=1.
    for j in range ( len(xm) ):
        if i != j:
            y*=( x-xm[j])/( xm[i]-xm[j])
    return y

#Interpolation
def lagrangeInterpolation (x , xm , ym ): 
    n=len( xm )
    lagrpoly = [ my_lagrange (x ,i , xm ) for i in range ( n )]
    y = np.dot( ym , lagrpoly ) #Vector's multiplication
    return y

def interpolateFunction(new_x, xm, ym):
    new_y = [lagrangeInterpolation(x, xm, ym) for x in new_x]
    return new_y

## Newtone interpolation

In [7]:
def _poly_newton_coefficient(x,y):
    """
    x: list or np array contanining x data points
    y: list or np array contanining y data points
    """

    m = len(x)

    x = np.copy(x)
    a = np.copy(y)
    for k in range(1,m):
        a[k:m] = (a[k:m] - a[k-1])/(x[k:m] - x[k-1])

    return a

def newton_polynomial(x_data, y_data, x):
    """
    x_data: data points at x
    y_data: data points at y
    x: evaluation points
    """
    a = _poly_newton_coefficient(x_data, y_data)
    n = len(x_data) - 1 # Degree of polynomial
    p = a[n]
    for k in range(1,n+1):
        p = a[n-k] + (x -x_data[n-k])*p
    return p

## Second lab simple print

In [8]:
def printTable(x, y, y_newt, y_lagr):
    print("x   ", "f(x)   ", "P(x)   ", "L(x)")
    for i in range(len(x)):
        print(x[i], y[i], y_newt[i], y_lagr[i])

        


## Third lab print

In [9]:
def printTable3(n):
    i0 = left_rectangles_rule(calcFunValue, -1, 1, n)
    i1 = trapezoid_rule(calcFunValue, -1, 1, n)
    i2 = simpson_rule(calcFunValue, -1, 1, n)
    print("N=", n, " I0=", i0, " I1=", i1, " I2=", i2)

# Integration

In [10]:
def integrate(f, a, b, p, method):
    
    # Spliting 
    # p means precision
    
    n = 2
    result1 = method(f, a, b, n)
    n *= 2
    result2 = method(f, a, b, n)
    
    while abs(result1 - result2) > p:
        n *= 2
        result1 = method(f, a, b, n)
        n *= 2
        result2 = method(f, a, b, n)
    return result2

## Integration by rectangles (left)

In [11]:
def left_rectangles_rule(f, a, b, n):
    offset = (b-a)/float(n)  #Calculate offset with current N
    total = sum([f((a + (k * offset))) for k in range(0, n)])   #Sum of f(x) from offsetted values 
    result = offset * total   #Multiply rusult to iffset value (look at lefr rectangles formula)
    return result

## Trapezoid integration

In [12]:
def trapezoid_rule(f, a, b, n):
    dx = 1.0 * (b - a) / n # Calculate offset
    sum = 0.5 * (f(a) + f(b))
    for i in range(1, n):
        sum += f(a + i * dx)

    return sum * dx

## Simpson integration

In [13]:
def simpson_rule(f, a, b, n):
    
    if n % 2 == 1:
        n += 1  #Even parabols number only
    dx = (b - a) / n
    sum = (f(a) + 4 * f(a + dx) + f(b))
    for i in range(1, (int)(n / 2)):
        sum += 2 * f(a + (2 * i) * dx) + 4 * f(a + (2 * i + 1) * dx)
    return sum * dx / 3

# Cauchy issue solving

In [14]:
def euler_method(function, n, xlist, ylist):
    h = 1/n
    
    for i in range(1, n):
        ylist[i] = ylist[i-1] + h * function(xlist[i-1], ylist[i-1])
    return ylist

In [15]:
def runge_kutta_method(function, n, xlist, ylist):
    h = 1/n

    for i in range(1, n):
        _y = ylist[i-1] + h * function(xlist[i-1], ylist[i-1])
        ylist[i] = ylist[i-1] + h * ((function(xlist[i-1], ylist[i-1]) + function(xlist[i], _y))/2)
    return ylist

In [16]:
def f3(x, y):
    return y * cos(x)

def f3_true(x):
    return math.e ** math.sin(x) # exact solution

#Known solution
x0 = 0
y0 = 1

In [17]:
def printTable4(xlist, ylist_tilda, ylist):
    for i in range(len(xlist)):
        print(xlist[i], ylist_tilda[i], ylist[i], abs(ylist[i] - ylist_tilda[i]))

# /////////////////////////////////////////////////Examples//////////////////////////////////////////////////////////

## Lab1 & Lab2

In [20]:
x_values = split(-1, 1, 5)
y_values = valuesInPonts(x_values)

# Obtain x values on the vector. The last one is 'M'
x_new_values = split(-1, 1, 6) 

#Calculate true y values
y_new_true_values = valuesInPonts(x_new_values) 

# Calculate interpolated values based on lagrange
y_new_Lagrange_values = interpolateFunction(x_new_values, x_values, y_values) 

# Calculate interpolated values based on Newtonr
y_new_Newtone_values = [newton_polynomial(x_values, y_values, x) for x in x_new_values] 

#Printing
printTable(x_new_values, y_new_true_values, y_new_Newtone_values, y_new_Lagrange_values)

x    f(x)    P(x)    L(x)
-1.0 1.306932828523934 1.306932828523934 1.306932828523934
-0.6 1.0220934246704843 1.025572018701671 1.025572018701671
-0.19999999999999996 1.0000319824255361 0.9981233465464665 0.9981233465464664
0.20000000000000018 1.0000319824255361 0.9981233465464665 0.9981233465464667
0.6000000000000001 1.0220934246704843 1.0255720187016713 1.0255720187016708
1.0 1.306932828523934 1.306932828523934 1.306932828523934


## Lab3

In [21]:
integrate(calcFunValue, -1, 1, eps, left_rectangles_rule)

2.1060199234189003

In [22]:
integrate(calcFunValue, -1, 1, eps, trapezoid_rule)

2.1060199234189003

In [23]:
integrate(calcFunValue, -1, 1, eps, simpson_rule)

2.1060199195705387

In [24]:
printTable3(1000)

N= 1000  I0= 2.1060207453674433  I1= 2.1060207453674433  I2= 2.1060199203421273


## Lab 4

In [25]:
n = 1000 # N*
h = 1/n

a,b = x0, 1

x_values = split(a, b, n)
y_values = np.zeros(n)
y_values[0] = y0

###Manage comments to use one or 

#y_values_tilda = euler_method(f3, n, x_values, y_values)
y_values_tilda = runge_kutta_method(f3, n, x_values, y_values)

y_values = [f3_true(x) for x in x_values]

printTable4(x_values, y_values_tilda, y_values)

0.0 1.0 1.0 0.0
0.001001001001001001 1.0010004997492488 1.0010015020023775 1.0022531287123826e-06
0.002002002002002002 1.0020019994942375 1.0020040060059998 2.0065117622980466e-06
0.003003003003003003 1.0030044992284413 1.003007512006339 3.0127778978261688e-06
0.004004004004004004 1.0040079989423105 1.0040120199958391 4.0210535285911675e-06
0.005005005005005005 1.005012498623264 1.0050175299639073 5.0313406432245245e-06
0.006006006006006006 1.0060179982556805 1.0060240418969073 6.043641226805008e-06
0.007007007007007007 1.0070244978208902 1.00703155577815 7.0579572597484486e-06
0.008008008008008008 1.0080319972971676 1.0080400715878866 8.074290718917965e-06
0.009009009009009009 1.0090404966597226 1.009049589303299 9.092643576291692e-06
0.01001001001001001 1.0100499958806932 1.0100601088984935 1.0113017800295054e-05
0.011011011011011011 1.011060494929137 1.0110716303444918 1.1135415354690537e-05
0.012012012012012012 1.0120719937710234 1.0120841536092222 1.2159838198799733e-05
0.01301301

0.3973973973973974 1.4720158046025713 1.4725857518577592 0.0005699472551878859
0.3983983983983984 1.4733734482664091 1.473945281647665 0.0005718333812558818
0.3993993993993994 1.4747317714043757 1.4753054931107275 0.0005737217063517264
0.40040040040040037 1.4760907722245185 1.476666384453228 0.0005756122287094989
0.4014014014014014 1.4774504489292524 1.4780279538758 0.0005775049465477355
0.4024024024024024 1.4788107997153577 1.4793901995734284 0.0005793998580707616
0.4034034034034034 1.4801718227739769 1.4807531197354442 0.0005812969614673591
0.4044044044044044 1.481533516290612 1.482116712545524 0.0005831962549118774
0.40540540540540543 1.4828958784451227 1.4834809761816865 0.0005850977365637888
0.4064064064064064 1.4842589074117227 1.4848459088162898 0.0005870014045670224
0.4074074074074074 1.4856226013589786 1.48621150861603 0.0005889072570512965
0.4084084084084084 1.4869869584498066 1.4875777737419371 0.0005908152921305643
0.4094094094094094 1.48835197684147 1.4889447023493743 0.00

0.8208208208208209 2.0771020937026474 2.0786228118447654 0.0015207181421179605
0.8218218218218218 2.0785176107425127 2.08004078391666 0.0015231731741471854
0.8228228228228228 2.079932567596725 2.081458195797316 0.0015256282005911004
0.8238238238238238 2.0813469603893258 2.0828750436022427 0.0015280832129169752
0.8248248248248248 2.082760785241597 2.084291323444177 0.0015305382025800895
0.8258258258258259 2.08417403827208 2.0857070314331034 0.0015329931610232883
0.8268268268268268 2.0855867155965933 2.08712216367627 0.001535448079676538
0.8278278278278278 2.0869988133282473 2.0885367162782056 0.0015379029499582586
0.8288288288288288 2.0884103275774657 2.0899506853407397 0.0015403577632739918
0.8298298298298298 2.0898212544520005 2.091364066963018 0.0015428125110172886
0.8308308308308309 2.0912315900569505 2.092776857241521 0.0015452671845705979
0.8318318318318318 2.0926413304947786 2.09418905227008 0.0015477217753012695
0.8328328328328328 2.09405047186533 2.0956006481398983 0.0015501762